In [1]:
import pandas as pd
import numpy as np
from matplotlib import colors
from matplotlib import rc
import matplotlib.pyplot as plt

from matplotlib import rcParams

latex_path = '/usr/local/texlive/2023/bin/universal-darwin'

rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = r'\usepackage{{amsmath}}\usepackage{{amsfonts}}\usepackage{{amssymb}}'
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Computer Modern Roman']
rcParams['mathtext.fontset'] = 'cm'

In [2]:

file_name = 'CCHU_NIRS.xlsx'
df = pd.read_excel(file_name)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df = df[['Subject_ID','probe', 'Block1 (S1)' ,'Filter Used for Block 1 (S1)','Block2 (S1)','Filter Used for Block 2 (S1)','Block1 (S2)','Filter Used for Block 1 (S2)', 'Block2 (S2)',
                'Filter Used for Block 2 (S2)', 'Block1 (S3)','Filter Used for Block 1 (S3)', 'Block2 (S3)','Filter Used for Block 2 (S3)']]

In [4]:
df

,Subject_ID,probe,Block1 (S1),Filter Used for Block 1 (S1),Block2 (S1),Filter Used for Block 2 (S1),Block1 (S2),Filter Used for Block 1 (S2),Block2 (S2),Filter Used for Block 2 (S2),Block1 (S3),Filter Used for Block 1 (S3),Block2 (S3),Filter Used for Block 2 (S3)
0,NC301,CCHUNB6A,1084.0,0.3,2103.0,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NC301,CCHUNB6A,1084,0.5,2103,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NC302,CCHUNB6A,1084,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NC302,CCHUNB6A,1084,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NC303,CCHUNB6A,1084,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2343,NE300,CCHUNB6A,1084,0.2,2103,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2344,NE300,CCHUNB6A,1084,0.3,2103,0.3,1084,0.3,2103,0.4,NaN,NaN,NaN,NaN
2345,NE300,CCHUNB6A,1084,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2346,NE300,CCHUNB6A,1084,0.3 Good for A & B; No Filter Good for C & D,2103,0.3 Good for A & B; No Filter Good for C & D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print('Different block names for Block 1 (S1)')
print(df['Block1 (S1)'].unique())
df.loc[df['Block1 (S1)'] == '1084', 'Block1 (S1)'] = float(1084)
df.loc[df['Block1 (S1)'] == '2101', 'Block1 (S1)'] = float(2101)
df.loc[df['Block1 (S1)'] == '2102', 'Block1 (S1)'] = float(2102)
df = df[df['Block1 (S1)'] != '-']; df = df[df['Block1 (S1)'] != '-8'];

print(df['Block1 (S1)'].unique())


Different block names for Block 1 (S1)
[1084.0 '1084' nan 2101.0 2102.0 '2101' '2102' 2103.0 '-' '-8']
[1084.0 nan 2101.0 2102.0 2103.0]


In [6]:
for b in df['Block1 (S1)'].unique():
    print('Block name '+str(b) + ' has '+str(len(df[df['Block1 (S1)']==b])) + ' rows')


Block name 1084.0 has 2282 rows
Block name nan has 0 rows
Block name 2101.0 has 29 rows
Block name 2102.0 has 8 rows
Block name 2103.0 has 1 rows


#### In the upcoming code the rows for the different blocks are filtered for A,B,C distances. Remaining only the D distances.
#### There are also values like 0.4 ; 0.2 which aren't taken into account because we don't know the distances

In [7]:
# Identifying how many rows have the D distance and extracting that value, filtering A, B and C distances. 

filters_1084 = np.array(df['Filter Used for Block 1 (S1)'][df['Block1 (S1)']== 1084.0])
filters_1084_float = filters_1084.copy()
rejected_elements = []
found_number = False
for i,f in enumerate(filters_1084_float): 
    try:
        if any(letter in f for letter in ['A', 'B', 'C']):
            #print('i: ' +str(i))

            if 'D' in f:
                found_number = False
                try: 
                    index = f.index(' D')
                except:
                    try: 
                        index = f.index('CD')
                    except: 
                        index = f.index(',D')
                    
                while found_number == False:
                    #print(f[index-3:index])
                    if any(char in f[index-3:index] for char in ['No', 'NO', 'no',]):
                        found_number = True
                        filters_1084_float[i] = np.nan
                        rejected_elements = np.append(rejected_elements,str(i) +' '+f)
                    elif '0' in f[index-3:index]: 
                        found_number = True
                        print(fr'Element {i}  :  {f} -- extracted value:  {f[index-3:index]}')
                        filters_1084_float[i] = float(f[index-3:index])
                    else: index = index - 1 
                
            else: 
                rejected_elements = np.append(rejected_elements,str(i) +' '+f)
                filters_1084_float[i] = np.nan

        elif ';' in f:
            filters_1084_float[i] = np.nan
            rejected_elements = np.append(rejected_elements,str(i) +' '+f)


    except: 
        pass

#filters_1084_float = filters_1084_float[~np.isnan(filters_1084_float)]


Element 52  :  0.5Good for ABC, 0.3GOOD FOR D -- extracted value:  0.3
Element 143  :  1.0 Good for A, 0.7 Good for B, 0.2 Good for C & D -- extracted value:  0.2
Element 144  :  0.8 Good for A & B, 0.3 Good for C & D -- extracted value:  0.3
Element 145  :  0.1 Good for C and D, 0.6 Good for A and B -- extracted value:  0.1
Element 146  :  0.7 Good for A and B, 0.1 Good for C and D -- extracted value:  0.1
Element 147  :  0.7 Good for A and B, 0.1 Good for C and D -- extracted value:  0.1
Element 232  :  0.3 Good for A, B and C, 0.1 Good for C and D -- extracted value:  0.1
Element 233  :  0.3 Good for A and B, 0.1 Good for C and D -- extracted value:  0.1
Element 317  :  0.3 Good for A, B, and C, 0.1 Good for C and D -- extracted value:  0.1
Element 382  :  0.8 Good for A and B; 0.4 Good for C and D -- extracted value:  0.4
Element 383  :  0.6 Good for A, B and C, 0.2. Good for D -- extracted value:  0.2
Element 384  :  0.6 good for A, B & C detectors, 0.5 good for C and D detectors 

In [8]:
print(fr'{len(rejected_elements)} rejected rows corresponding to A, B, & C distances')

42 rejected rows corresponding to A, B, & C distances


### Manual processing of data

In [9]:
for i,filt  in enumerate(filters_1084_float): 
    try:
        if any(letter in filt for letter in ['nd', '/','No Filter','No filter', ',','..',"'0.4'"]):
            print(fr'index {i}:  {filters_1084_float[i]}')
    except:
        pass

index 77:  0.4 nd
index 789:  0.3 for first block msmt,0.2 for second
index 794:  1.1/0.7
index 932:  No filter
index 1053:  No filters noted
index 1443:  0.7, 0.4
index 1465:  0.4 nd
index 1477:  0..9
index 1727:  No Filter
index 1746:  No Filter
index 1904:  0.7, 0.3
index 1923:  0.3 nd
index 2204:  0.2 nd


In [10]:
filters_1084_float[77] = 0.4
filters_1084_float[789] = 0.3
filters_1084_float[1465] = 0.4
filters_1084_float[1923] = 0.3
filters_1084_float[2204] = 0.2
filters_1084_float = np.delete(filters_1084_float,1904)
filters_1084_float = np.delete(filters_1084_float,1746)
filters_1084_float = np.delete(filters_1084_float,1727)
filters_1084_float = np.delete(filters_1084_float,1477)
filters_1084_float = np.delete(filters_1084_float,1443)
filters_1084_float = np.delete(filters_1084_float,1053)
filters_1084_float = np.delete(filters_1084_float,932)
filters_1084_float = np.delete(filters_1084_float,794)


In [12]:
arr_float = np.zeros(len(filters_1084_float))
arr_str = filters_1084_float
for i, elem in enumerate(arr_str):
    try:
        arr_float[i] = float(elem)
    except ValueError:
        # Handle cases where conversion fails
        print("Element '{}' at index {} could not be converted to float.".format(elem, i))
        arr_float[i] = np.nan  # Or any other appropriate value

print("Array as float:", arr_float)

Element 'O.4' at index 2019 could not be converted to float.
Array as float: [0.3 0.5 0.3 ... 0.3 nan 0.2]


In [13]:
filters_1084_float[2019] = 0.4
filters_1084_float = filters_1084_float.astype(float)
filters_1084_float = filters_1084_float[~np.isnan(filters_1084_float)]

In [18]:
len(filters_1084_float)

2221

In [58]:
t = df['Block1 (S1)'].unique()
t

array([1084.0, '1084', nan, 2101.0, 2102.0, '2101', '2102', 2103.0, '-',
       '-8'], dtype=object)

In [59]:
type(t[0])

float

In [12]:
#df[df['Block1 (S1)']=='1084.0']

In [9]:
filter_B1_S1 = np.array(df['Filter Used for Block 1 (S1)']); 

In [15]:
df_1084_b1 = df[df['Block1 (S1)']=='1084']
df_1084_b1['probe'].unique()

array(['CCHUNB6A', 'CCHUNB1', 'CCHUNB3', 'CCHUNB2'], dtype=object)

In [54]:
filters = np.array(df_1084_b1['Filter Used for Block 1 (S1)'])

'0.5'

In [29]:
df['Block1 (S1)'].unique()

array([1084.0, '1084', nan, 2101.0, 2102.0, '2101', '2102', 2103.0, '-',
       '-8'], dtype=object)

In [48]:
#df[df['Block1 (S1)']==1084]